In [ ]:
import pandas as pd
import numpy as np
import sklearn
import os
from matplotlib import pyplot as plt

In [ ]:
os.listdir("../input/")

In [ ]:
filepath = "../input/train.csv"

In [ ]:
crica = pd.read_csv(filepath,
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [ ]:
crica

In [ ]:
listNull = [x for x in [(row,crica[row].isnull().sum()) for row in crica] if x[1]!=0]
print(listNull)
for row in crica.drop("Id",axis='columns'):
    print(row, crica[row].value_counts(), sep='\n')

In [ ]:
crica.drop(["v2a1", "v18q1", "rez_esc", "SQBmeaned", "meaneduc"], axis='columns', inplace=True) #NaN
crica["idhogar"] = crica["idhogar"].transform(lambda x: int(x,16))
print(crica["idhogar"].value_counts())
crica["dependency"] = crica["dependency"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
print(crica["dependency"].value_counts())
crica["edjefe"] = crica["edjefe"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
print(crica["edjefe"].value_counts())
crica["edjefa"] = crica["edjefa"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
print(crica["edjefa"].value_counts())
#crica.drop(["dependency", "edjefe", "edjefa", "idhogar", "Id"], axis='columns', inplace=True) #strings
#crica.drop(["dependency", "edjefe", "edjefa", "Id"], axis='columns', inplace=True) #strings
crica.drop(["idhogar", "Id"], axis='columns', inplace=True) #strings

In [ ]:
nCrica = crica.dropna()

In [ ]:
nCrica

In [ ]:
testfilepath = "../input/test.csv"

In [ ]:
testCrica = pd.read_csv(testfilepath,
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [ ]:
testCrica

In [ ]:
listNull = [x for x in [(row,testCrica[row].isnull().sum()) for row in testCrica] if x[1]!=0]
print(listNull)

In [ ]:
testCrica.drop(["v2a1","v18q1", "rez_esc", "SQBmeaned", "meaneduc"], axis='columns', inplace=True) #Nan
testCrica["idhogar"] = testCrica["idhogar"].transform(lambda x: int(x,16))
testCrica["dependency"] = testCrica["dependency"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
testCrica["edjefe"] = testCrica["edjefe"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
testCrica["edjefa"] = testCrica["edjefa"].transform(lambda x: 1 if x=="yes" else 0 if x=="no" else x)
#testCrica.drop(["dependency", "edjefe", "edjefa", "idhogar"], axis='columns', inplace=True) #strings
#testCrica.drop(["dependency", "edjefe", "edjefa"], axis='columns', inplace=True) #strings
testCrica.drop("idhogar", axis='columns', inplace=True) #strings

In [ ]:
nTestCrica = testCrica

In [ ]:
nTestCrica

In [ ]:
Ycrica = nCrica.Target

In [ ]:
Xcrica = nCrica.drop("Target", axis="columns")

In [ ]:
Xcrica

In [ ]:
XtestCrica = nTestCrica.drop("Id", axis='columns')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(knn, Xcrica, Ycrica, cv=10)

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
knn.fit(Xcrica,Ycrica)

In [ ]:
YtestPred = knn.predict(XtestCrica)

In [ ]:
YtestPred

In [ ]:
savepath = "results.csv"

In [ ]:
#YtestPred.to_csv(savepath)
#import numpy as np
#np.savetxt("foo.csv", YtestPred, delimiter=",")
ypanda = pd.DataFrame(YtestPred, columns = ["Target"])
totalypanda = nTestCrica.combine_first(ypanda)
totalypanda["Target"] = totalypanda["Target"].transform(lambda x: int(x))
saveYpanda = totalypanda[["Id", "Target"]]
saveYpanda.to_csv(savepath, index=False)
saveYpanda